In [1]:
import pandas as pd

In [2]:
reforecast_df = pd.read_csv("/Users/giulioelmi/Desktop/kelshi_trading/reforecast_data.csv")
weather_df = pd.read_csv("/Users/giulioelmi/Desktop/kelshi_trading/final_weather_data.csv")

In [3]:
reforecast_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1674 entries, 0 to 1673
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1674 non-null   int64  
 1   date_utc    1674 non-null   object 
 2   asof_utc    1674 non-null   object 
 3   tmax_K      1674 non-null   float64
 4   tmax_C      1674 non-null   float64
 5   tmax_F      1674 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 78.6+ KB


In [4]:
reforecast_df.head()

,Unnamed: 0,date_utc,asof_utc,tmax_K,tmax_C,tmax_F
0,0,2021-01-01,2020-12-31T20:00:00Z,291.28,18.13,64.634
1,1,2021-01-02,2021-01-01T20:00:00Z,290.31,17.16,62.888
2,2,2021-01-03,2021-01-02T20:00:00Z,288.16,15.01,59.018
3,3,2021-01-04,2021-01-03T20:00:00Z,288.55,15.40,59.720
4,4,2021-01-05,2021-01-04T20:00:00Z,290.26,17.11,62.798


In [5]:
reforecast_df = (
    reforecast_df
        .rename(columns={
            "asof_utc": "DATE",
            "tmax_F": "forecasted_TMAX",
        })
)

reforecast_df["DATE"] = (
    pd.to_datetime(reforecast_df["DATE"], utc=True)
    .dt.strftime("%Y-%m-%d")
)

In [6]:
reforecast_df["DATE"] = pd.to_datetime(reforecast_df["DATE"])
reforecast_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1674 entries, 0 to 1673
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Unnamed: 0       1674 non-null   int64         
 1   date_utc         1674 non-null   object        
 2   DATE             1674 non-null   datetime64[ns]
 3   tmax_K           1674 non-null   float64       
 4   tmax_C           1674 non-null   float64       
 5   forecasted_TMAX  1674 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 78.6+ KB


In [7]:
print(reforecast_df["DATE"].max())
print(reforecast_df["DATE"].min())

2025-07-31 00:00:00
2020-12-31 00:00:00


In [8]:
end = "2025-07-31"
sliced_reforecast = reforecast_df[(reforecast_df["DATE"] <= end)].copy()
sliced_reforecast = sliced_reforecast[["DATE", "forecasted_TMAX"]]
sliced_reforecast.tail()


,DATE,forecasted_TMAX
1669,2025-07-27,70.826
1670,2025-07-28,72.122
1671,2025-07-29,73.364
1672,2025-07-30,73.508
1673,2025-07-31,73.004


In [9]:
weather_df["DATE"] = pd.to_datetime(weather_df["DATE"])
weather_df.tail()

,DATE,TMAX,TMIN,PRCP,AWND,WDF2,WSF2,year
1668,2025-07-27,72,62,0.0,8.95,260.0,17.0,2025
1669,2025-07-28,73,63,0.0,8.28,240.0,18.1,2025
1670,2025-07-29,74,63,0.0,7.16,250.0,21.0,2025
1671,2025-07-30,74,63,0.0,7.16,260.0,18.1,2025
1672,2025-07-31,73,63,0.0,7.61,270.0,16.1,2025


In [10]:
training_data = weather_df.merge(sliced_reforecast, how = 'left', on = 'DATE')

In [11]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1673 entries, 0 to 1672
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   DATE             1673 non-null   datetime64[ns]
 1   TMAX             1673 non-null   int64         
 2   TMIN             1673 non-null   int64         
 3   PRCP             1673 non-null   float64       
 4   AWND             1672 non-null   float64       
 5   WDF2             1673 non-null   float64       
 6   WSF2             1673 non-null   float64       
 7   year             1673 non-null   int64         
 8   forecasted_TMAX  1673 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(3)
memory usage: 117.8 KB


In [12]:
training_data.to_csv("/Users/giulioelmi/Desktop/kelshi_trading/training_data.csv", index=False)